In [3]:
!pip install opencv-python numpy gradio mediapipe Pillow moviepy

In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp
from PIL import Image, ImageOps, ImageDraw
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import random
import math

# Initialisation
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_face_and_eyes(image_rgb):
    """
    Détection robuste avec validation stricte
    """
    h, w, _ = image_rgb.shape
    img_for_detection = image_rgb.astype(np.uint8)

    results = []

    # Méthode 1: MediaPipe - Plus précise
    try:
        rgb_image = cv2.cvtColor(img_for_detection, cv2.COLOR_BGR2RGB)
        res = mp_face_mesh.process(rgb_image)

        if res and res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark

            # Points clés pour les yeux
            left_eye_points = [
                (lm[33].x * w, lm[33].y * h),   # Coin externe
                (lm[133].x * w, lm[133].y * h), # Coin interne
                (lm[160].x * w, lm[160].y * h), # Bas
                (lm[158].x * w, lm[158].y * h), # Haut
            ]

            right_eye_points = [
                (lm[362].x * w, lm[362].y * h), # Coin interne
                (lm[263].x * w, lm[263].y * h), # Coin externe
                (lm[385].x * w, lm[385].y * h), # Bas
                (lm[387].x * w, lm[387].y * h), # Haut
            ]

            # Centre des yeux (moyenne des points)
            left_eye = np.mean(left_eye_points, axis=0)
            right_eye = np.mean(right_eye_points, axis=0)

            # Validation géométrique
            eye_distance = np.linalg.norm(right_eye - left_eye)
            eye_center_y = (left_eye[1] + right_eye[1]) / 2

            # Critères de validation
            if (50 < eye_distance < w * 0.7 and  # Distance raisonnable
                0.1 * h < eye_center_y < 0.7 * h):  # Position verticale réaliste

                results.append({
                    'left_eye': left_eye,
                    'right_eye': right_eye,
                    'confidence': 0.95,
                    'method': 'MediaPipe',
                    'eye_distance': eye_distance
                })

    except Exception as e:
        print(f"MediaPipe error: {e}")

    # Méthode 2: Haar Cascade
    try:
        gray = cv2.cvtColor(img_for_detection, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(100, 100))

        if len(faces) > 0:
            # Prendre le plus grand visage
            fx, fy, fw, fh = max(faces, key=lambda f: f[2] * f[3])

            # Estimation anatomique des yeux
            face_center_x = fx + fw / 2
            eye_y = fy + fh * 0.37  # Position réaliste des yeux
            eye_separation = fw * 0.25  # Distance réaliste

            left_eye = np.array([face_center_x - eye_separation, eye_y])
            right_eye = np.array([face_center_x + eye_separation, eye_y])

            results.append({
                'left_eye': left_eye,
                'right_eye': right_eye,
                'confidence': 0.75,
                'method': 'Haar_Cascade',
                'eye_distance': eye_separation * 2,
                'face_rect': (fx, fy, fw, fh)
            })

    except Exception as e:
        print(f"Haar Cascade error: {e}")

    if not results:
        return None

    # Retourner le meilleur résultat
    best = max(results, key=lambda x: x['confidence'])
    return best

def align_eyes_to_fixed_position(image_rgb, target_left, target_right, output_size=(1280, 720)):
    """
    Aligne les yeux à des positions ABSOLUES sans zoom excessif
    """
    detection = detect_face_and_eyes(image_rgb)

    if detection is None:
        print("❌ PAS DE VISAGE DÉTECTÉ - Image ignorée")
        return None

    current_left = detection['left_eye']
    current_right = detection['right_eye']
    method = detection['method']

    print(f"👁️  Détection: {method}")
    print(f"   Yeux actuels: L{current_left.astype(int)} R{current_right.astype(int)}")

    # Calculer les transformations nécessaires
    current_center = (current_left + current_right) / 2.0
    target_center = (np.array(target_left) + np.array(target_right)) / 2.0

    # 1. ROTATION pour horizontaliser les yeux
    current_vector = current_right - current_left
    current_angle = math.atan2(current_vector[1], current_vector[0])
    target_angle = math.atan2(target_right[1] - target_left[1], target_right[0] - target_left[0])
    rotation_needed = target_angle - current_angle

    print(f"🔄 Rotation nécessaire: {math.degrees(rotation_needed):.2f}°")

    # 2. ÉCHELLE - Contrôlée et limitée
    current_eye_distance = np.linalg.norm(current_vector)
    target_eye_distance = np.linalg.norm(np.array(target_right) - np.array(target_left))
    scale_factor = target_eye_distance / current_eye_distance if current_eye_distance > 0 else 1.0

    # LIMITATION DU ZOOM - Éviter les zooms extrêmes
    scale_factor = np.clip(scale_factor, 0.5, 2.0)  # Limiter entre 50% et 200%

    print(f"📏 Échelle appliquée: {scale_factor:.3f}")

    # 3. Appliquer la transformation étape par étape
    h, w = image_rgb.shape[:2]

    # Étape 1: Rotation autour du centre des yeux actuels
    M_rot = cv2.getRotationMatrix2D(tuple(current_center), math.degrees(rotation_needed), scale_factor)
    rotated = cv2.warpAffine(image_rgb, M_rot, (w, h),
                            flags=cv2.INTER_LANCZOS4,
                            borderMode=cv2.BORDER_CONSTANT,
                            borderValue=(255, 255, 255))

    # Calculer la nouvelle position des yeux après rotation
    current_eyes_homog = np.array([[current_left[0], current_right[0]],
                                   [current_left[1], current_right[1]],
                                   [1, 1]])

    M_rot_3x3 = np.vstack([M_rot, [0, 0, 1]])
    rotated_eyes = M_rot_3x3 @ current_eyes_homog
    new_center = np.mean(rotated_eyes[:2, :], axis=1)

    # Étape 2: Translation vers la position cible
    translation = target_center - new_center
    print(f"➡️  Translation: {translation.astype(int)}")

    M_trans = np.float32([[1, 0, translation[0]], [0, 1, translation[1]]])
    final_result = cv2.warpAffine(rotated, M_trans, output_size,
                                 flags=cv2.INTER_LANCZOS4,
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue=(255, 255, 255))

    return final_result

def create_aligned_slideshow(files, music=None, duration_per_face=4.0, fade_duration=1.5):
    """
    Créer un diaporama avec alignement strict des yeux
    """
    if not files:
        return "Aucune image fournie.", None, None

    os.makedirs('outputs', exist_ok=True)
    clips = []

    # POSITIONS FIXES ET ABSOLUES
    output_size = (1280, 720)

    # Position optimale pour les yeux (centre horizontal, légèrement haut)
    center_x = output_size[0] // 2
    eye_y = 250  # Position verticale
    eye_separation = 140  # Distance entre les yeux

    FIXED_LEFT_EYE = (center_x - eye_separation//2, eye_y)
    FIXED_RIGHT_EYE = (center_x + eye_separation//2, eye_y)

    print(f"🎯 POSITIONS CIBLES FIXES:")
    print(f"   Œil gauche: {FIXED_LEFT_EYE}")
    print(f"   Œil droit: {FIXED_RIGHT_EYE}")
    print(f"   Écartement: {eye_separation}px")

    successful_images = 0

    for i, file in enumerate(files):
        try:
            print(f"\n📸 === TRAITEMENT IMAGE {i+1}/{len(files)} ===")
            print(f"Fichier: {os.path.basename(file.name)}")

            # Charger l'image
            pil_img = Image.open(file.name).convert('RGBA')

            # Rotation légère pour effet vintage (très minime)
            rotation = random.uniform(-0.5, 0.5)  # Rotation minimale
            if rotation != 0:
                pil_img = pil_img.rotate(rotation, expand=True, resample=Image.BICUBIC)

            # Fond blanc
            white_bg = Image.new("RGBA", pil_img.size, (255, 255, 255, 255))
            white_bg.paste(pil_img, (0, 0), pil_img)
            white_bg = white_bg.convert('RGB')

            # Bordure discrète
            bordered_pil = ImageOps.expand(white_bg, border=10, fill='white')

            # Conversion numpy
            img_np = np.array(bordered_pil)

            # ALIGNEMENT
            aligned_img = align_eyes_to_fixed_position(
                img_np,
                FIXED_LEFT_EYE,
                FIXED_RIGHT_EYE,
                output_size
            )

            if aligned_img is not None:
                # Créer le clip vidéo
                clip = ImageClip(aligned_img).set_duration(duration_per_face)
                clips.append(clip)
                successful_images += 1

                # Sauvegarder pour debug
                debug_path = f'outputs/aligned_{i+1:02d}.jpg'
                cv2.imwrite(debug_path, cv2.cvtColor(aligned_img, cv2.COLOR_RGB2BGR))

                print(f"✅ Image {i+1} alignée avec succès")
            else:
                print(f"⚠️  Image {i+1} IGNORÉE (pas de visage détecté)")

        except Exception as e:
            print(f"❌ Erreur image {i+1}: {e}")
            continue

    if not clips:
        return "❌ Aucune image avec visage détecté. Vérifiez vos images.", None, None

    if successful_images < len(files):
        print(f"⚠️  {len(files) - successful_images} image(s) ignorée(s) (pas de visage)")

    print(f"\n🎬 CRÉATION VIDÉO avec {successful_images} images")

    # Créer la vidéo finale
    final_clip = concatenate_videoclips(clips, padding=-fade_duration, method="compose")

    # Ajouter la musique
    if music is not None:
        try:
            audio = AudioFileClip(music.name).set_duration(final_clip.duration)
            final_clip = final_clip.set_audio(audio)
            print("🎵 Audio ajouté")
        except Exception as e:
            print(f"⚠️  Erreur audio: {e}")

    # Sauvegarder
    output_path = 'outputs/aligned_slideshow.mp4'
    print("💾 Sauvegarde de la vidéo...")

    final_clip.write_videofile(output_path, fps=24, codec='libx264', audio_codec='aac',
                              ffmpeg_params=['-pix_fmt', 'yuv420p'], verbose=False, logger=None)

    final_clip.close()  # Libérer la mémoire

    summary = f"""✅ VIDÉO CRÉÉE AVEC SUCCÈS!

📊 Résumé:
• Images traitées: {successful_images}/{len(files)}
• Images ignorées: {len(files) - successful_images} (pas de visage)
• Position des yeux: FIXE à {FIXED_LEFT_EYE} et {FIXED_RIGHT_EYE}
• Zoom limité: Entre 50% et 200% pour éviter les déformations

📁 Fichiers créés:
• Vidéo finale: {output_path}
• Images debug: outputs/aligned_XX.jpg

💡 Conseil: Seules les images avec des visages clairement détectables sont incluses."""

    return summary, output_path, output_path

# Interface Gradio optimisée
iface = gr.Interface(
    fn=create_aligned_slideshow,
    inputs=[
        gr.File(label="📸 Images (uniquement celles avec des visages)",
                file_types=["image"],
                file_count="multiple"),
        gr.File(label="🎵 Musique (optionnel)",
                file_types=["audio"]),
        gr.Slider(minimum=2.0, maximum=8.0, value=4.0, step=0.5,
                 label="⏱️ Durée par image (secondes)"),
        gr.Slider(minimum=0.5, maximum=3.0, value=1.5, step=0.5,
                 label="🔄 Durée transition (secondes)")
    ],
    outputs=[
        gr.Textbox(label="📋 Rapport détaillé", lines=10),
        gr.Video(label="🎬 Vidéo finale"),
        gr.File(label="💾 Télécharger")
    ],
    title="🎯 Alignement Parfait des Yeux - Version Robuste",
    description="""
    **ALIGNEMENT ULTRA-PRÉCIS avec GESTION INTELLIGENTE** 🎯

    ✅ **Détection robuste**: Ignore automatiquement les images sans visage
    ✅ **Positions absolues**: Yeux aux mêmes coordonnées exactes
    ✅ **Zoom contrôlé**: Limité entre 50%-200% pour éviter les déformations
    ✅ **Rotation précise**: Yeux parfaitement horizontaux
    ✅ **Debug intégré**: Images de vérification sauvegardées

    ⚠️ **Important**: Seules les images avec des visages clairement détectés seront incluses dans la vidéo finale.
    """,
    allow_flagging='never'
)

if __name__ == "__main__":
    iface.launch(share=True, debug=True)


  warnings.warn(



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://56b6328e24a0e95982.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎯 POSITIONS CIBLES FIXES:
   Œil gauche: (570, 250)
   Œil droit: (710, 250)
   Écartement: 140px

📸 === TRAITEMENT IMAGE 1/4 ===
Fichier: Image 1.jpg


  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



👁️  Détection: MediaPipe
   Yeux actuels: L[812 581] R[935 610]
🔄 Rotation nécessaire: -13.52°
📏 Échelle appliquée: 1.110
➡️  Translation: [-233 -345]
✅ Image 1 alignée avec succès

📸 === TRAITEMENT IMAGE 2/4 ===
Fichier: Image 2.jpg
👁️  Détection: Haar_Cascade
   Yeux actuels: L[449 673] R[501 673]
🔄 Rotation nécessaire: 0.00°
📏 Échelle appliquée: 2.000
➡️  Translation: [ 165 -423]
✅ Image 2 alignée avec succès

📸 === TRAITEMENT IMAGE 3/4 ===
Fichier: Image 3.jpg
👁️  Détection: MediaPipe
   Yeux actuels: L[559 522] R[698 557]
🔄 Rotation nécessaire: -14.03°
📏 Échelle appliquée: 0.977
➡️  Translation: [  11 -290]
✅ Image 3 alignée avec succès

📸 === TRAITEMENT IMAGE 4/4 ===
Fichier: Image 4.jpg
👁️  Détection: Haar_Cascade
   Yeux actuels: L[633 559] R[747 559]
🔄 Rotation nécessaire: 0.00°
📏 Échelle appliquée: 1.223
➡️  Translation: [ -50 -309]
✅ Image 4 alignée avec succès

🎬 CRÉATION VIDÉO avec 4 images
💾 Sauvegarde de la vidéo...
